# ICPSR Dataset Archive

In [11]:
import pandas as pd
import difflib
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Label
from networkx.readwrite import json_graph
import ipywidgets as widgets
import json

import RecommendationSystem as rs
import NetworkBuilder as nb

pd.set_option('display.max_colwidth', -1)

datasets = pd.read_json('data/train_test/data_sets.json')
publications = pd.read_json('data/train_test/publications.json')
datasets['title_lower'] = datasets['title'].str.lower()
publications['title_lower'] = publications['title'].str.lower()

def readNX(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

G = nb.buildG()
G = nb.addCommunity(G)
# G = readNX('data/network_v2.5_contract.json')

In [8]:
@interact

def showDatasets(By = ['Dataset Title', 'Dataset ID', 'Publication Title', 'Publication ID', 'DOI'], Search="ANES 1952"):
    column = {
        'Dataset Title': 'title_lower',
        'Dataset ID': 'data_set_id',
        'Publication Title': 'title_lower',
        'Publication ID': 'publication_id',
        'DOI': 'unique_identifier'
    }
    slist = Search.lower().split(',')
#     return slist
    try:
        if(By == 'Dataset Title'):
            return datasets[['data_set_id', 'unique_identifier','title', 'description']].loc[datasets[column[By]].str.contains('|'.join(slist))]
        elif (By == 'Dataset ID'):
            tmp = datasets[['data_set_id', 'unique_identifier','title', 'description']].loc[datasets[column[By]].isin([int(n) for n in slist])]
            sorter = pd.DataFrame([int(n) for n in slist], columns=['data_set_id'])
            return pd.merge(sorter, tmp, how='left', on='data_set_id')
        elif (By == 'Publication Title'):
            return publications[['publication_id', 'unique_identifier','title']].loc[publications[column[By]].str.contains('|'.join(slist))]
        elif (By == 'Publication ID'):
            tmp = publications[['publication_id', 'unique_identifier','title']].loc[publications[column[By]].isin([int(n) for n in slist])]
            sorter = pd.DataFrame([int(n) for n in slist], columns=['publication_id'])
            return pd.merge(sorter, tmp, how='left', on='publication_id')
        elif (By == 'DOI'):
            sl
            tmp = datasets[['data_set_id', 'unique_identifier','title', 'description']].loc[datasets[column[By]].str.contains('|'.join(slist))]
            tmp2 = publications[['publication_id', 'unique_identifier','title']].loc[publications[column[By]].str.contains('|'.join(slist))]
            return tmp if tmp.shape[0]>0 else tmp2            
    except Exception as e:
        print("ERROR "+e)
        pass
    

interactive(children=(Dropdown(description='By', options=('Dataset Title', 'Dataset ID', 'Publication Title', …

In [9]:
category = ['Keyword', 'Dataset', 'Publication Paper']
metric = ['Cosine', 'Jaccard', 'Adamic-Adar', 'Hopcroft']

@interact
def showRecommendations(category=typeDD, Search='', Metric=metric, live=False):
    if live:
        return rs.getRecommendations(Search_By, Search, Metric, G)

interactive(children=(Dropdown(description='Search_By', options=('Keyword', 'Dataset', 'Publication Paper'), v…